<a href="https://colab.research.google.com/github/probabll/ntmi-tutorials/blob/main/T2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guide

Check the guide carefully before starting.

## ILOs

After completing this lab you should be able to 

* understand NBC's factorisation, parameterisation, parameter estimation, and key algorithms
* implement a Naive Bayes text classifier
* predict and evaluate models using precision/recall

**General notes**

* In this notebook you are expected to use $\LaTeX$. 
* Use python3.
* Use NLTK (3.5) to read annotated data.




## Table of contents

* [Labelled Data](#data)
* [Theory](#theory)  
    * [Text Classifier](#textcls)
    * [Generative Classifier](#gencls)
        * [Naive Bayes Classifier](#NBC)
        * [Classification](#classification)
        * [Parameter Estimation](#MLE)
* [Implementation](#code) 
* [Experiment](#exp) 
* [Further analysis](#further) 



## Table of graded exercises

Exercises have equal weights.

* [Data](#ex-data)
* [NBC](#ex-NBC)
* [Grid search](#ex-grid)
* [Error analysis](#ex-analysis)


## How to use this notebook

* Check the entire notebook before you get started, this gives you an idea of what lies ahead.
* Note that, as always, the notebook recaps theory, and contains solved exercises. While you should probably make use of this theory recap, be careful not to spend disproportionately more time on this than you should. The theory here is more condensed, and should be easier to understand after reading the corresponding prep material for HC2a and after the highlights discussed in class (HC2a).
* I recommend you read the theory part before the LC2 session, aim at 1h investment, then work on the coding part during the LC, and then invest another 2h of self-study (for example, to finish up the notebook or to just review it).
* The last section is optional for this tutorial, but note that it helps connect T1 and T2. So, you might want to work on it once you are done with the graded exercises.


# <a name="data"> Labelled Data

In this tutorial we will be looking into text classification with labelled data (that is, text that has been categorised or somehow labelled for certain attributes). 
    
This can be binary classification, for example, 

* `nltk.corpus.sentence_polarity`
* `nltk.corpus.movie_reviews`
* `nltk.corpus.subjectivity`    
    
One multiclass classification, for example, 
* `nltk.corpus.brown`    

In [ ]:
# You might need to install these:


# !pip install pandas
# !pip install seaborn
# !pip install nltk
# !pip install tabulate
# !pip install sklearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('sentence_polarity')
nltk.download('movie_reviews')
nltk.download('subjectivity')
nltk.download('brown')

It's always good to visualise some properties of the data we will be manipulating. Basic properties of interest are the frequency of the classes, the lenght of the documents in it. 

Visualising the frequency of classes can warn you that your dataset is unballanced, length of documents can warn you that some datasets can be challenging for you personal computer.

We could plot these quantities any way we like, but pandas and seaborn together make it really easy to get insightful visualisations, so we will share this trick with you.

In [ ]:
def get_corpus_df(corpus, categories=None):
    """
    Return a tall dataframe for a nltk text classification corpus.
    corpus: an instance of nltk.corpus meant for text classification
    categories: None or a list of categories (in case we want to use just a portion of the labelled documents)
    """
    if categories is None:
        categories = corpus.categories()
    rows = []
    for label in categories:
        rows.extend((label, len(x)) for x in corpus.sents(categories=[label]))
    return pd.DataFrame(rows, columns=['label', 'length'])

**Subjectivity**

In [ ]:
from nltk.corpus import subjectivity

subjectivity_df = get_corpus_df(subjectivity)

In [ ]:
_ = sns.catplot(y='label', orient='h', kind='count', data=subjectivity_df).set(title="subjectivity")

In [ ]:
_ = sns.catplot(x='length', y='label', kind='violin', data=subjectivity_df).set(title="subjectivity")

Two types of documents (objective and subjective) that are equally represented, and they are reasonably short documents (mostly 10-40 tokens). The distribution of document length is mostly similar for both classes, but the objective class shows a longer tail.

It is also a good idea to inspect actual documents in each class. Familiarity with our datasets makes us better researchers.

In [ ]:
for x, y in zip(subjectivity.sents(categories=['obj']), ['obj'] * 10):
    print(y, ' '.join(x))

In [ ]:
for x, y in zip(subjectivity.sents(categories=['subj']), ['subj'] * 10):
    print(y, ' '.join(x))

<a name="ex-data"> **Graded Exercise - Data** 
    
Use the same 3 visualisation/inspections techniques as well as 1 more (which you can choose) on the following three corpora:

* `nlt.corpus.sentence_polarity`
* `nltk.corpus.movie_reviews`
* `nltk.corpus.brown`

and like we did, make some remarks about what you see. For `brown` which has many classes, you can inspect fewer documents per class, or pick 3-4 of the classes and inspect documents for those only.

In [ ]:
# CONTRIBUTE YOUR SOLUTION

In this tutorial, **we will focus on sentiment classification** using the `sentence_polarity` dataset.

In [ ]:
from tabulate import tabulate
from collections import Counter

# <a name="theory"> Theory

We suggest you work through this section prior to the live session, so you can make the most of the live session in terms of coding exercises.
    
This section recaps the theory, if you completed prep-work for HC2a, you could even skip it. 

Note that this section contains many fact-checking exercises with solution, these can be useful to test whether your initial understanding of the theory and, generally, for self-study. 

## <a name="textcls"> Text Classifiers

In NLP we often want to design systems that can read a piece of text and categorise it as an instance of one of a finite set of classes. We call those systems *text classifiers* (or text categorisers).
    
It is seldom the case that a piece of text can be reasonably mapped to *a single category*, for various reasons that we have already discussed: language is ambiguous, we often lack context (about the situation, the writer, the reader), and many implicit attributes of text are subjective by nature (e.g., one's opinion about a product). So, in practice we face text classification as a pipeline: we combine a *statistical model* and a *decision rule*.
    
The statistical model realises the mapping from a given piece of text to a *conditional probability distribution* (cpd) over the categories available. The decision rule uses the information in this distribution to elect a single category that is shown to the user.
    
---

Consider the case of *sentiment classification* with three sentiment levels (i.e., negative, neutral, and positive). 

A **probabilistic sentiment classifier** maps a document $x \in \mathcal X$ to a probability distribution over the sample space $\mathcal Y = \{-1, 0, +1\}$, where -1 stands for negative, 0 stands for neutral, and +1 stands for positive sentiment. Note that we call $x$ a document, but it could be a sentence, a paragraph, or any granularity we like. A document $x$ is a sequence of words $x=\langle w_{1}, \ldots, w_{l} \rangle$, or $w_{1:l}$ for short, where $l = |x|$ is the sequence length. Each word belongs to a vocabulary $\mathcal W$ of known words, and the vocabulary size is $V$. 
The space of all possible documents $\mathcal X$ is the space of all sequences made of words from $\mathcal W$, and these sequences can be of any size, a set like that is denoted $\mathcal W^*$.

So, formally, we have a random variable $X$ taking on documents in the set $\mathcal X$, a random variable $Y$ taking on classes in the set $\mathcal Y$, and a random variable $W$ taking on words in a vocabulary $\mathcal W$. A statistical model then establishes a map from any one $x \in \mathcal X$ to the cpd $P_{Y|X=x}$.

Why does it make sense to see a classifier as a probability distribution? Well, there are a few technical reasons (we are generally very good at learning probability distributions, whereas other types of functions may be more difficult to learn), but here is an intuitive reason: probability distributions capture uncertainty due to the data (e.g., noise), due to the task definition (e.g., the task is subjective), or due to our inability to model the true underlying cognitive processes behind language understanding. See the example below.

We may say `no way` 
* in response to news about a global pandemic, in which case, we probably mean to express a negative sentiment about it;
* in response to our friend's startup being listed amongst the top-10 startups in the Netherlands, in which case we probably mean to express a positive sentiment about it;
* in response to a classmate asking `will you skip next NTMI's class?`, and in this case `no way` might just be an objective `no`, without a negative or positive sentiment.

If all we are given is the text `no way` and no additional context, nor any information about the speaker or the situation, then we cannot really determine the sentiment (not even as humans). This illustrates ambiguity/uncertainty that is inherent to the task/data. The best we can do is to pack a few assumptions into a model, use some data to estimate this model, and then let this model quantify the probability with which `no way` might express `negative`, `neutral`, or `positive` sentiment. 

As we said in the course, the probability of a certain sentiment $y \in \{-1, 0, 1\}$ given a piece of text $x$ is not a real attribute of $x$ in the world. Rather, is is a quantification of the uncertainty that a *model of sentiment analysis* can make given text and given all of the design assumptions we made. 


So, let's **design** one such model. We will pick sentiment analysis as the example, but other text classification problems fit in the same framework as long as the target classes come from a finite set of disjoint categories (e.g., sentiments, or topics, or types of named-entities, or spam vs not spam, or product categorisation in online shops, etc). Technically such sets are called *countably finite sets*.


**Goal** We want a conditional distribution $P_{Y|X=x}$ that, given some piece of text $x$ (e.g., a sentence, a paragraph, or a document), quantifies our model's beliefs in each of the classes in the sample space $\mathcal Y$.

**Challenge** Think about it for a moment, we need a distribution over a finite number of classes, let's say we have $K$ classes. This should not be too difficult right? We have already learnt about the Categorical distribution, so we could start by attempting to use something like 

\begin{align}
    Y | X=x &\sim \mathrm{Cat}(\boldsymbol\theta^{(x)}) \qquad \text{with }\boldsymbol\theta^{(x)} \in \Delta_{K-1}
\end{align}

That is, for each $x$, we have a $K$-dimensional probability vector $\boldsymbol\theta^{(x)}$ where $P_{Y|X}(y|x) = \theta^{(x)}_y$ is the probability, under this model, that $x$ should be labelled with $y$.
Recall that we called this a *tabular* representation of the Categorical cpd, the name comes from the fact that the collection of parameters necessary to prescribe all the Categorical distributions in this definition can be store in something that looks like a table (see the last section of T1).

**Exercises with solution** What is the problem with attempting to use a tabular representation of the conditional distributions $P_{Y|X}$?

<details>
    <summary><b>SOLUTION</b></summary>

The space of all documents $\mathcal X$ is practically infinite, if every conditioning document $x$ requires its own cpd with its own independent parameter vector, we will need infinitely many parameter vectors. This could never be stored. Even if we knew a clever data structures that can store such object efficiently, we could never estimate infinitely many vectors reliably, since we will only ever observe finitely many data points.
    
We might tell ourselves, well let's use *only* the documents that we have seen during training. In that case, we would never be able to classify a new document, and analysing new documents is the whole point of text classification in NLP.
    
---
    
</details>



There are at least two different approaches to this problem. The *generative* approach, which we discuss in *this* notebook, and the *discriminative* approach, which you will encounter in T3.


## <a name="gencls"> Generative Classifiers

A generative classifier is built upon a *joint distribution* $P_{YX}$ over the joint sample space $\mathcal Y \times \mathcal X$. We can use this model to *infer* a distribution $P_{Y|X=x}$ over target classes for any given $X=x$, and we can do so *on demand* (that is, whenever we observe a certain document $x$) without ever storing (and thus never having to estimate) one probability value for each and every possible pair $(x, y) \in \mathcal X \times \mathcal Y$.

**Exercise with solution** Suppose $\mathcal X$ is the space of all documents (all of them, no matter the length). Suppose $\mathcal Y = \{-1, 0, 1\}$. Explain in words what the joint sample space $\mathcal X \times \mathcal Y$ consists of and give 2 examples of outcomes in it. How large this space is?







<details>
    <summary><b>SOLUTION</b></summary>
    
The joint sample space is made of the cross product of the two sets. As the set of all documents is infinite, the joint sample space is also infinite. The elements in it are pairs of the kind $(x, y)$ where $x\in \mathcal X$ is a document, and $y \in \mathcal Y$ is one of $3$ categories. Here are some examples of outcomes in it
    
* (`i love dogs`, +1)
* (`i love dogs`, 0)
* (`i love dogs`, -1) 
    
Note that all outcomes are in there, whether or not they are plausible, even something as unreasonable as (`cats are better than dogs`, +1) is in there, as well as ill-formed sentences (`i am i am i`, +1).
    
---
    
</details>



It should not be obvious that using a joint distribution $P_{YX}$ rather than a collections of cpds $P_{Y|X}$ will make the problem any simpler. In fact, if we were to store one probability value per assignment $(X=x, Y=y)$, we would end up with the exact same problem as before (the tables would be much too large, we would be able to store them, and we would be able to estimate them effectively). 

But, a joint distribution, as it turns out, gives us an opportunity: to make simplifying assumptions about how the probability of an assignment *decomposes* in terms of probabilities of outcomes simpler than whole documents. The technical term here is **factorisation**, we will *factorise* the probability of $(X=x, Y=y)$ using cpds involving simpler variables (or rather, variables whose sample spaces are *finite* and hopefully much smaller).

<details>
    <summary><b>Factorising</b></summary>
    Factorise means to decompose using a product. You've done this before. For example, we can factorise the expression $ab + ac$ into $a\times (b +c)$.
    
You have also factorised in terms of pre-specified factors, for example, factorise 12 using prime numbers: $12 = 2\times2\times3$.
    
This is more or less what we will be doing: we will define some elementary *factors* and express probability values using those simpler factors. For us, the factors themselves are probability values, but they concern events from a smaller event space. The rules that allow us to recombine the simpler factor in order to answer whatever probability query are the rules of probability (i.e., chain rule, conditional probability, and marginal probability).
    
---
    
</details>

To specify a joint distribution that's tractable to represent and estimate, we start with the **chain rule**. 

**Joint distribution (first attempt)** 

\begin{align}
    P_{YX}(y, x) &= P_Y(y)P_{X|Y}(x|y)
\end{align}

This first attempt factors the probability of $y$ first, and then the probability of generating $x$ conditioned on $y$ being its label. This may seem counterintuitive at first, but imagine given that $y=-1$ you may indeed generate text that's very particular to negative sentiment, and not all too similar to what you would generate had $y$ been $+1$. Of course, there are exceptions to this, like our illustration with `no way`, but the intuition is still fairly reasonable. 

This factorisation does not *really* nail the problem just yet, but not that it does achieve something interesting. Whereas there are infinitely many cpds of the kind $P_{Y|X=x}$, one for each possible value of $x \in \mathcal X$, there are only $K$ cpds of the kind $P_{X|Y=y}$, one for each possible value of $y \in \mathcal Y$. If we can figure out a way to deal with the fact that the sample space of $P_{X|Y=y}$ is infinite (that is, we can generate any one of an inifinite set of possible documents), we should be able to get to a feasible design.

We achieved something here: we only need to specify $|\mathcal Y|$ cpds. But we are still stuck with the fact that a tabular representation for each one of the $K$ cpds of $P_{X|Y=y}$ would remain a very large object.

This time the *complex* outcome is on the generating side of the probability distribution (not on the conditioning side), so let's see if chain rule can help us once more:

\begin{align}
    P_{X|Y}(w_{1:l}|y) &= \prod_{i=1}^l P_{W|YH}(w_i|y, w_{<i})
\end{align}

where the random variable $W$ denotes a random word and the random variable $H$ denotes the history of random words from left-to-right (by assumption we go from left-to-right, but any order is valid under chain rule). The notation $w_{<i}$ denotes the sequence of words before the $i$th word.

This application of chain rule does a good thing and a bad thing. First, it simplifies the generating side (each factor now is responsible for assigning probability to 1 word in context), which is good for the sample space $\mathcal W$ is finite (it has $V$ words in it). But, it complicates the conditioning side again by pushing the history into it, and the problem with the history is that it's potentially very long (and it grows longer as we advance $i$ along the document). So now again we have to estimate a very large number of cpds: one for each possible value of the conditioning context $(y, w_{<i})$ in $P_{W|YH}$.

It looks like we cannot avoid having to deal with a very large space of options. And indeed, *without* making simplifying assumptions we cannot. 

But, these two applications of chain rule did achieve something remarkable, they show us that we can express the probability value of a labelled document $P_{YX}(y, x)$ using a product of other probability values, each probability value for a random variable with finite sample space.

\begin{align}
    P_{YX}(y, w_{1:l}) &= P_Y(y) \times \prod_{i=1}^l P_{W|YH}(w_i|y, w_{<i})
\end{align}

**Exercise with solution** What simplifying assumption could we make about how words depend on one another in a document in order to simplify the conditional factors $P_{X|YH}$?

<details>
    <summary><b>SOLUTION</b></summary>
    
We used chain rule to break the complex piece of text into smaller pieces, now we can make conditional independence assumptions to make these smaller pieces themselves simpler cpds. 
    
For example, let's make the conditional independence assumption that any two words are independent of one another given the class.
    
Mathematically this is expressed as $W_i \perp W_j \mid Y=y$ for $i \neq j$ in $X=w_{1:l}$. 
    
And the factorisation becomes:    
    
\begin{align}
    P_{YX}(y, w_{1:l}) &\overset{\text{ind.}}{=} P_Y(y) \prod_{i=1}^l \underbrace{P_{W|Y}(w_i|y)}_{\text{simpler}}
\end{align}
    
*Remark I.* If you have a hard time understanding this bit, have a look at the prep-work for HC2a, you might have missed some important background there.
    
*Remark II.* We use $\overset{\text{ind.}}{=}$ when an equality only holds under some independence assumption. This is different from chain rule, for example, which always holds.  
    
---
    
</details>



So far so good, but the original problem was to assign a distribution over classes given a document. Perhaps you can already see how the joint distribution above will help us get to the conditional we wanted all along. If not, don't worry, read on :)

### <a name="NBC"> Naive Bayes Classifier
    
The naive Bayes classifier (NBC) is called that way because it makes a strong conditional independence assumption (that's the 'naive' part of the name) and because it uses Bayes rule to infer the conditional probability $P_{Y|X}(y|x)$ from the joint distribution $P_{YX}$.

    
NBC factorises the **joint probability** of $(y, x)$ as:
    
\begin{align}
    P_{YX}(y, w_{1:l}) &\overset{\text{ind.}}{=} P_Y(y) \prod_{i=1}^l P_{W|Y}(w_i|y)
\end{align}

The simpler factor $P_Y$ is modelled as a Categorical distribution over classes.
The other factors, $P_{W|Y=y}$ are each a Categorical cpd devined over $V$ possible words, so $P_{W|Y}$ is a collection of $K$ Categorical cpds.


Here is the complete generative story as well as the parameterisation in terms of Categorical cpds:
    
1. Sample a target class from a prior distribution: $Y \sim \mathrm{Cat}(\phi_1, \ldots, \phi_K)$
2. Condition on $y$, and sample a word for each position $i$ from the class-conditioned distribution $W|Y=y \sim \mathrm{Cat}(\boldsymbol\pi^{(y)})$ with $\boldsymbol\pi^{(y)} \in \Delta_{V-1}$ until you draw a stop symbol (i.e., end-of-sequence or EOS).


Concretely, this is the probability value that the model assigns to a pair $(y, x)$:
    
\begin{align}
    P_{YX}(y, x) &\overset{\text{NBC}}{=} \mathrm{Cat}(y|\phi_{1:C}) \prod_{i=1}^l \mathrm{Cat}(w_i|\pi^{(y)}_{1:v}) \\
    &= \phi_y \prod_{i=1}^l \pi^{(y)}_{w_i}
\end{align}

<details>
    <summary><b>Note on meaning of generative story</b></summary>
    
If the idea of a "generative story" confuses you, don't worry you are not alone. Let's try and clarify that. The story tells how we *assume* the observations came about, it does not tell us how the observations did come about.
    
We did not actually drew samples from this distribution and ended up with a dataset filled with perfectly valid labelled documents, rather, real people labelled documents and we just downloaded a digital version of their hardwork.
    
But, now that we have a story that could have generated the data, in the sense that the data are in the support of the joint probability distribution we describe, we can assess the probability with which we would have generated the observed data. This quantity has practical uses, for example, in maximum likelihood estimation we use this quantity to try and find the parameters $\boldsymbol\phi$ and $\boldsymbol\pi$ that maximise the probability of this model ever generating the observed data.
    
</details>    


<details>
    <summary><b>What's an end-of-sequence symbol?</b></summary>
    
As we shall see in the course, models based on generative stories often need a criterion to "stop" their generation process. This is relevant after training, when we sometimes use the model to generate data; and it is also relevant for training, in that this criterion may help the model learn something useful (for example, the length of a document or something like that). 
    
The end-of-sequence symbol is a special symbol that's added to the vocabulary and used to mark the end of a sentence. Common choices are `</s>` and `-EOS-` for these sequences are unlikely to belong to the vocabulary of any language, but you can choose anything you like (as long as it is not a word in your dataset).

Before we estimate the parameters of our model, we make sure to add this EOS symbol to the end of our sequences. In the future we may need other symbols, such as BOS (begin-of-sequence), etc.
    
</details>    

**Exercise with solution** Express the probability that the NBC assigns to (`no way`, +1) in terms of the parameters of the model? How about (`no , no way`, -1)?

<details>
    <summary><b>SOLUTION</b></summary>

*Advice.* Read the question carefully. Oftentimes people start trying to compute the MLE solution, and they get confused because we gave them no data. The question is not about parameter estimation. Rather, the question asks you to express the probability of those data points in terms of the parameters of the model (the numerical values of those parameters and where they come from is not important for this). 
    
First, we pad the document with EOS symbol, i.e., we treat the document as `no way EOS`, then we plug into the formula for NBC getting 
    
\begin{equation}
    \phi_{+1} \times \pi^{(+1)}_{\text{no}} \times \pi^{(+1)}_{\text{way}} \times \pi^{(+1)}_{\text{EOS}}
\end{equation}
    
It's similar for the other case,     
\begin{equation}
    \phi_{-1} \times \left(\pi^{(-1)}_{\text{no}} \right)^2 \times \pi^{(-1)}_{\text{,}} \times \pi^{(-1)}_{\text{way}} \times \pi^{(-1)}_{\text{EOS}}
\end{equation}    
but note that we use a different cpd, this time conditioning on -1 rather than +1, we also have the probability of the comma (given -1), and the probability of `no` (given -1) factors in twice.     
    
---
    
</details>


**Exercise with solution** Can we use the NBC to assign probability to a document $x$ for which we do not know the class? If so, explain how and compute the probability of `no way`, make sure to express it in terms of the parameters of the model. 

<details>
    <summary><b>SOLUTION</b></summary>
    
The NBC is a joint distribution $P_{YX}$, if we are given only one outcome $X=x$, and not the other, we can always use *marginalisation* to compute $P_{X}(x)$:
    
\begin{align}
    P_X(x) &= \sum_{y \in \mathcal Y} P_{YX}(y, x) \\
    &=  \sum_{y \in \mathcal Y} P_{Y}(y) \prod_{i=1}^l P_{W|Y}(w_i|y) \\
    &=  \sum_{y \in \mathcal Y} \phi_{y} \prod_{i=1}^l \pi^{(y)}_{w_i}
\end{align}
    
Then the probability of `no way` is
\begin{align}
    P_X(\langle \text{no}, \text{way}, \text{EOS} \rangle) 
    &= \phi_{-1} \times \pi^{(-1)}_{\text{no}} \times \pi^{(-1)}_{\text{way}} \times \pi^{(-1)}_{\text{EOS}} \\
    &+ \phi_{0} \times \pi^{(0)}_{\text{no}} \times \pi^{(0)}_{\text{way}} \times \pi^{(0)}_{\text{EOS}} \\
    &+\phi_{+1} \times \pi^{(+1)}_{\text{no}} \times \pi^{(+1)}_{\text{way}} \times \pi^{(+1)}_{\text{EOS}}
\end{align}
    

**Warning** If this exercise was challenging, check out the prep-work for HC2a, in particular, the introduction to probabilistic graphical models.
   
---
    
</details>



**Exercise with solution** How do we obtain a cpd $P_{Y|X=x}$ given some input text? Give the expression in terms of parameters of the model.

<details>
    <summary><b>SOLUTION</b></summary>
    
Via *Bayes rule*!
    
\begin{align}    
    P_{Y|X}(y|x) &= \frac{P_{YX}(y, x)}{P_{X}(x)} \\
    &= \frac{P_Y(y)\prod_{i=1}^l P_{W|Y}(w_i|y)}{P_{X}(x)} \\
\end{align}    

Note that we can compute all terms in this formula: $P_Y(y)$ is $\phi_y$, $P_{W|Y}(w_i|y)$ is $\pi^{(y)}_{w_i}$, and we've just solved $P_X(x)$ in the previous exercise.
   
</details>

---

### <a name="classification"> Classification

How do we actually classify using the NBC? That's a fair question, and if you solved the previous quiz, you sort of already found the answer.

The algorithm that maps from a probability distribution $P_{Y|X=x}$ to a single outcome is called a **decision rule**, literally because it is a made up rule that makes decisions for us. The most popular decision rule in text classification is the *most probable class* rule, which classifies a document $x$ with the label that receives highest probability under the model. 

Essentially, when we are given a new document $x_*$, we can solve the following search problem:


\begin{align}
y^* &= \mathrm{argmax}_{k\in \mathcal Y} ~ P_{Y|X}(k|x_*)
\end{align}

That is, we are looking for the class $k$ in $\mathcal Y$ that receives maximum *posterior probability* $P_{Y|X}(k|x_*)$, this is also called the *mode* of the distribution.
    
In fact, we can solve an even simpler problem if we apply the definition of conditional probabilities:
\begin{align}
y^* &= \mathrm{argmax}_{k\in \mathcal Y} ~ \frac{P_{YX}(k, x_*)}{P_X(x_*)}
\end{align}
as we are searching for the value of $k$ while keeping the document fixed, the denominator does not change the result and can be ignored:
\begin{align}
y^* &= \mathrm{argmax}_{k\in \mathcal Y} ~ P_{YX}(k, x_*)
\end{align}
    
This says, the most probable class a-posteriori (that is, after observing the document) is the class for which the joint probability $P_{YX}(k, x_*)$ is highest.

We can then substitute in our model definition for the joint probability    
\begin{align}
y^* &= \mathrm{argmax}_{k\in \mathcal Y} ~ P_{YX}(k, x_*) \\
    &= \mathrm{argmax}_{k\in \mathcal Y} ~ P_{Y}(k) \prod_{i=1}^{l_*} P_{W|Y}(w_i|k) \\
\end{align}    
and finally, substitute in our model parameters:
\begin{align}    
y^*    &= \mathrm{argmax}_{k\in \mathcal Y} ~  \phi_k \prod_{i=1}^{l_*} \pi_{w_i}^{(k)} \\
\end{align}
   

**Exercise with solution** If $\phi_{-1}=\phi_{+1} = \frac{2}{5}$, $\phi_0 = \frac{1}{5}$ and some of the parameters $\pi^{(y)}$ are given in the rows of the following table

| $Y$ | all | the | same | EOS | ... |
|-----|-----|-----|------|-----|-----|
| -1  | a   | c   | b    | d   | ... |
| 0   | a   | c   | 2.5b   | d   | ... |
| 1   | a   | c   | b    | d   | ... |

What is the most probable label of the document `no freaking way EOS`?

<details>
    <summary><b>Solution</b></summary>

Let's start by expressing the joint probabilities $P_{YX}(y, \text{all the same EOS})$ for each possible value of $y$:
* $y=-1$: $\frac{2}{5} \times a \times c\times b \times d = \frac{2}{5}abcd$
* $y=0$: $\frac{1}{5} \times a \times c \times 2.5b  \times d = \frac{2.5}{5}abcd$
* $y=+1$: $\frac{2}{5} \times a \times c \times b \times d = \frac{2}{5}abcd$

We should now pick the label that is assigned maximum probability when paired with this document, that would be $y=0$.

See that even though we had a prior preference in favour of labels other than neutral, given the evidence in the document, the posterior prefers the neutral label.
    
</details>    

### <a name="MLE"> Parameter Estimation

We know how NBC factorises, i.e., 

\begin{equation}
P_{YX}(y, w_{1:l}) = P_Y(y)\prod_{i=1}^l P_{W|Y}(w_i|y)
\end{equation}

we have chosen a parameterisation of our cpds, i.e., 

\begin{align}
P_{YX}(y, w_{1:l}) &= \mathrm{Cat}(y|\phi_{1:K})\prod_{i=1}^l \mathrm{Cat}(w_i|\pi^{(y)}_{1:V}) \\
&= \phi_{y} \prod_{i=1}^l \pi^{(y)}_{w_i}
\end{align}

now we need only estimate the numerical values of the parameters $\boldsymbol \phi$ and $\boldsymbol \pi$.

If we are given a dataset of $N$ labelled observations $\mathcal D = \{ (x^{(n)}, y^{(1)}), \ldots, (x^{(N)}, y^{(N)}) \}$, where each document $x^{(n)}$ has size $l_n$, we can do so via maximum likelihood estimation.

**Exercise with solution** What is the MLE for $\phi_{y}$ for some $y$ in $\mathcal Y$?

<details>
    <summary><b>SOLUTION</b></summary>
    
    
\begin{align}
\phi_y = \frac{\mathrm{count}_{Y}(y)}{\sum_{k \in \mathcal Y}\mathrm{count}_Y(k)} = \frac{\mathrm{count}_{Y}(y)}{N}
\end{align}
    
where we use $\mathcal D$ to determine the counts as shown below
    
\begin{equation}
    \mathrm{count}_Y(k) = \sum_{n=1}^N [y^{(n)} = k]
\end{equation}

For a *balanced* dataset, that is, a dataset with equal number of labelled instances for each class, $Y \sim \mathrm{Cat}(\phi_{1:K})$ becomes the uniform distribution, that is, $\phi_y = 1/K$ for all $y \in \mathcal Y$.

---    
    
</details>



**Quiz** What is the MLE for $\pi^{(y)}_w$ for some $y$ in $\mathcal Y$ and $w$ in $\mathcal W$?

<details>
    <summary><b>SOLUTION</b></summary>
    
    
\begin{align}
\pi^{(y)}_{w} = \frac{\mathrm{count}_{YX}(y, w)}{\sum_{v \in \mathcal W}\mathrm{count}_{YX}(y, v)}
\end{align}
    
where we use $\mathcal D$ to determine the counts as shown below
    
\begin{equation}
    \mathrm{count}_{YX}(k, v) = \sum_{n=1}^N \sum_{i=1}^{l_n} [y^{(n)} = k] \times [w^{(n)}_{i} = v]
\end{equation}
    
    
</details>

---

# <a name="code"> Implementation

    
We will design a python class that implements NBC, but before we get to code, let's check if you are getting comfortable with the formal presentation of the model (which is in principle enough to answer the following quizzes).

**Quiz** What is the time complexity of assessing joint probability $P_{YX}(y, w_{1:l})$?

<details>
    <summary><b>SOLUTION</b></summary>
    
It takes $\mathcal O(l)$, since we have to assess $P_Y(y)$ once and $P_{W|Y=y}$ once for every word in $w_{1:l}$.

---
    
</details>



**Quiz** What is the time complexity of assessing marginal probability $P_X(w_{1:l})$?

<details>
    <summary><b>SOLUTION</b></summary>
    
It takes $\mathcal O(K \times l)$, since we need to assess the joint $P_{YX}(y, w_{1:l})$ once for every label in the sample space $\{1, \ldots, K\}$. This is necessary so we can marginalise $Y$ out.
    
---
    
</details>



**Quiz** What is the storage requirements of the NBC model?

<details>
    <summary><b>SOLUTION</b></summary>
    
It requires space dominated by $\mathcal O(K \times V)$ since we have to store a $K$-dimensional vector $\phi_{1:K}$ and $K$ class-conditioned cpds, each defined over $V$ words.
    
---
    
</details>



**Quiz** What is the time complexity to classify $w_{1:l}$ with a trained model?

<details>
    <summary><b>SOLUTION</b></summary>
    
To classify, we need to assess the joint probability $P_{YX}(y, x_{1:l})$ once for each label in the sample space $\{1, \ldots, K\}$, and then pick the one which is maximum. This takes time $\mathcal O(K \times l)$ since each assessment of joint probability is linear in $l$ and finding the maximum in a list is not worse than linear.
    
---
    
</details>



In [ ]:
def validate_categorical_cpd(params: dict):
    """
    Just to remind you a valid Categorical cpd is a parameterised by a vector of positive 
    numbers that sum to 1.
    
    We normally think of the parameters as a sequence or a vector, 
    but in a programming language, it can be useful to use a dict, 
    with a dict the indexing does not need to be numerical (we could use the label 'name' 
    or the word itself to index positions in the dictionary).
    
    In some cases a dict may be considered slow compared to a list or array, 
    but for the scale of our experiments in the tutorial, it's fine and 
    conceptually much easier.
    
    params: map outcome to probability mass
    """
    return all(0 <= p <= 1 for p in params.values()) and np.isclose(sum(p for p in params.values()), 1., 1e-4)

In [ ]:
assert validate_categorical_cpd({'pos': 0.2, 'neg': 0.8})  # fine
assert validate_categorical_cpd({'pos': 0.3, 'neg': 0.7})  # fine
assert validate_categorical_cpd({'pos': 0.2, 'neu': 0.1, 'neg': 0.7})  # we can have more than 2, that's fine 
assert validate_categorical_cpd({'pos': 0.2, 'neu': 0.0, 'neg': 0.8})  # we can have 0 probs inside
assert not validate_categorical_cpd({'pos': 0.2, 'neg': 0.7})  # not good
assert not validate_categorical_cpd({'pos': 0.2})  # not good
assert not validate_categorical_cpd({'pos': -0.2, 'neu': 0.2, 'neg': 0.8})  # we cannot have 'negative probs' inside

We will now begin implementing a little piece at a time all functionalities necessary to get NBC off the floor.

As we will be manipulating some tables, the first two functions are already implemented for you to show you how you can use python dictionaries to store tables.


**Exercise with solution** study and understand the design of `get_prior_parameter` and `get_cond_parameter` below.

In [ ]:
def get_prior_parameter(y: str, phi: dict):
    """
    Return P(Y=y) under the model Categorical(phi).
    
    y: the label
    phi: a dictionary that maps from the label to its probability mass
        even though mathematically we think of phi as a vector, 
        in code it can be convenient to treat it like a dictionary some times, 
        for example, as a dictionary we can use labels that are strings (rather than 0-based indices)
        
        For this implementation assume that phi has already been validated as a Categorical parameter.
        
    Return: Categorical(y|\phi)
    """
    # **SOLUTION**
    return phi.get(y, 0)  # if an outcome is outside the support, it gets 0 mass

In [ ]:
assert get_prior_parameter('pos', {'pos': 0.2, 'neg': 0.8}) == 0.2
assert get_prior_parameter('neg', {'pos': 0.3, 'neg': 0.7}) == 0.7
assert get_prior_parameter('neu', {'pos': 0.3, 'neg': 0.7}) == 0.

In [ ]:
def get_cond_parameter(y: str, w: str, pi: dict, UNK=None):
    """
    Return P(W=w|Y=y) under the model Categorical(\pi^{(y)}).
    
    y: the label
    w: word
    pi: a dictionary of dictionaries 
        first we can index it using a label to obtain a dict as return, 
        the latter dict are the parameters for a distribution over the vocabulary given the label
        and it can be indexed using a word to obtain a probability mass

        even though mathematically we think of pi as a table/matrix
        in code it can be convenient to treat it like a dictionary some times, 
        for example, as a dictionary we can use labels that are strings (rather than 0-based indices)
        and words that are strings (rather than 0-based indices to a vocabulary).        
        
        For this implementation assume that every dict inside of pi
         has already been validated as a Categorical parameter.
    UNK: reserved for future use
        
    Return: Categorical(w|\phi^{(y)})
    """
    # **SOLUTION**
    cpd = pi.get(y, None)
    if cpd is None:  # the label is not in the support of the model
        return 0. 
    return cpd.get(w, 0.)  # if a word is outside the support, it gets 0 mass

In [ ]:
# A test case for us, the cpds in it are valid as we can see
# (do not change this test case, it will be used in other cells)
test_phi_1 = {'pos': 0.6, 'neg': 0.4}
test_pi_1 = {
    'pos': {'not': 0.05, 'so': 0.1, 'good': 0.5, 'bad': 0.15, 'okay': 0.2 }, 
    'neg': {'not': 0.05, 'so': 0.1, 'good': 0.1, 'bad': 0.5, 'okay': 0.25 }
}
assert validate_categorical_cpd(test_phi_1)
assert all(validate_categorical_cpd(cpd) for y, cpd in test_pi_1.items())

# Fine
assert get_cond_parameter('pos', 'good', test_pi_1) == 0.5
assert get_cond_parameter('pos', 'not', test_pi_1) == 0.05
assert get_cond_parameter('pos', 'so', test_pi_1) == 0.1
assert get_cond_parameter('pos', 'bad', test_pi_1) == 0.15
assert get_cond_parameter('pos', 'okay', test_pi_1) == 0.2

# Fine
assert get_cond_parameter('neg', 'good', test_pi_1) == 0.1
assert get_cond_parameter('neg', 'not', test_pi_1) == 0.05
assert get_cond_parameter('neg', 'so', test_pi_1) == 0.1
assert get_cond_parameter('neg', 'bad', test_pi_1) == 0.5
assert get_cond_parameter('neg', 'okay', test_pi_1) == 0.25

# Not in the support
assert get_cond_parameter('neu', 'okay', test_pi_1) == 0.
assert get_cond_parameter('pos', 'strange', test_pi_1) == 0.
assert get_cond_parameter('neg', 'strange', test_pi_1) == 0.
assert get_cond_parameter('neu', 'strange', test_pi_1) == 0.

**Exercise with solution** Impement a function to return $\log P_Y(y)$ for a label $y \in \mathcal Y$ under this model. Your function should use the functionality `get_prior_parameter` and/or `get_cond_parameter` from earlier. Assume the model parameters are already available. 

Below you will find some assertions (ie, test cases) for you to verify your implementation.

See API below:

In [ ]:
def log_prior_prob(y: str, phi: dict):
    """Return log P(Y=y) under the model"""
    raise NotImplemented("Implement me!")

<details>
    <summary><b>Solution</b></summary>
    
```python
def log_prior_prob(y: str, phi: dict):
    """Return log P(Y=y) under the model"""
    # **SOLUTION**
    return np.log(get_prior_parameter(y, phi))    
```
    
---    
</details>

In [ ]:
# Your code should pass these tests
assert log_prior_prob('pos', {'pos': 0.2, 'neg': 0.8}) == np.log(0.2)
assert log_prior_prob('neg', {'pos': 0.3, 'neg': 0.7}) == np.log(0.7)

**Exercise with solution** Impement a function to return $\log P_{X|Y}(x|y)$ for a label $y \in \mathcal Y$  and document $x \in \mathcal X$ under this model. Your function should use the functionality `get_prior_parameter` and/or `get_cond_parameter` from earlier. Assume the model parameters are already available. 

Below you will find some assertions (ie, test cases) for you to verify your implementation.

See API below:

In [ ]:
def log_conditional_prob(x: list, y: str, pi: dict):
    """Return log P(X=x|Y=y)"""
    raise NotImplemented("Implement me!")

<details>
    <summary><b>Solution</b></summary>
    
```python
def log_conditional_prob(x: list, y: str, pi: dict):
    """Return log P(X=x|Y=y) under the model"""
    # **SOLUTION**
    # named arguments help us not make mistakes (for example, flipping y and w accidentaly)
    return sum(np.log(get_cond_parameter(y=y, w=w, pi=pi)) for w in x)      
```
    
---    
</details>

In [ ]:
# Your code should pass these tests

# Fine
assert np.isclose(log_conditional_prob("not bad".split(), 'pos', test_pi_1), np.log(0.05 * 0.15), 1e-3)
assert np.isclose(log_conditional_prob("not good".split(), 'pos', test_pi_1), np.log(0.05 * 0.5), 1e-3)
assert np.isclose(log_conditional_prob("good".split(), 'pos', test_pi_1), np.log(0.5), 1e-3)

assert np.isclose(log_conditional_prob("not bad".split(), 'neg', test_pi_1), np.log(0.05 * 0.5), 1e-3)
assert np.isclose(log_conditional_prob("not good".split(), 'neg', test_pi_1), np.log(0.05 * 0.1), 1e-3)
assert np.isclose(log_conditional_prob("good".split(), 'neg', test_pi_1), np.log(0.1), 1e-3)

A good code should also deal with cases that *are not* in the support of the joint distribution. Those should have 0 probability for now (later we discuss a technique to avoid 0 probs). The log of 0 is -inf.

In [ ]:
# Not in the support (should produce warnings, but pass the test)
assert log_prior_prob('neu', {'pos': 0.3, 'neg': 0.7}) == -np.inf
assert log_conditional_prob("not that bad".split(), 'pos', test_pi_1) == -np.inf
assert log_conditional_prob("not that bad".split(), 'neg', test_pi_1) == -np.inf
assert log_conditional_prob("not okay".split(), 'neu', test_pi_1) == -np.inf

**Exercise with solution** Impement a function to return $\log P_{YX}(y, x)$ for a label $y \in \mathcal Y$  and document $x \in \mathcal X$ under this model. Your function should use the functionality `log_prior_prob` and/or `log_conditional_prob` from earlier. Assume the model parameters are already available. 

Below you will find some assertions (ie, test cases) for you to verify your implementation.

See API below:

In [ ]:
def log_joint_prob(y: str, x: list, phi: dict, pi: dict):
    """Return log P(Y=y, X=x) under the model"""
    raise NotImplemented("Implement me!")

<details>
    <summary><b>Solution</b></summary>
    
```python
def log_joint_prob(y: str, x: list, phi: dict, pi: dict):
    """Return log P(Y=y, X=x) under the model"""
    # **SOLUTION**
    # named arguments help us not make mistakes (eg, flipping x and y in the conditional)
    return log_prior_prob(y, phi) + log_conditional_prob(x=x, y=y, pi=pi)
```
    
---    
</details>

In [ ]:
# Your code should pass these tests

assert np.isclose(
    log_joint_prob(y='pos', x="not bad".split(), phi=test_phi_1, pi=test_pi_1), 
    np.log(0.6 * 0.05 * 0.15), 
    1e-3
)
assert np.isclose(
    log_joint_prob(y='pos', x="not good".split(), phi=test_phi_1, pi=test_pi_1), 
    np.log(0.6 * 0.05 * 0.5), 
    1e-3
)


assert np.isclose(
    log_joint_prob(y='neg', x="not bad".split(), phi=test_phi_1, pi=test_pi_1), 
    np.log(0.4 * 0.05 * 0.5), 
    1e-3
)
assert np.isclose(
    log_joint_prob(y='neg', x="not good".split(), phi=test_phi_1, pi=test_pi_1), 
    np.log(0.4 * 0.05 * 0.1), 
    1e-3
)

# NBC is not sensitive to the order of words within the sentence

assert np.isclose(
    log_joint_prob(y='pos', x="bad not".split(), phi=test_phi_1, pi=test_pi_1), 
    np.log(0.6 * 0.05 * 0.15), 
    1e-3
)

assert np.isclose(
    log_joint_prob(y='neg', x="good not".split(), phi=test_phi_1, pi=test_pi_1), 
    np.log(0.4 * 0.05 * 0.1), 
    1e-3
)

**Exercise with solution** Impement a function to return $\log P_{X}(x)$ for a document $x \in \mathcal X$ under this model. Your function should use the functionality `log_prior_prob` and/or `log_conditional_prob` and/or `log_joint_prob` from earlier. Assume the model parameters are already available. 

Below you will find some assertions (ie, test cases) for you to verify your implementation.

See API below:

In [ ]:
def log_marginal_prob(x: list, phi: dict, pi: dict):
    """Return log P(X=x) under the model"""    
    raise NotImplemented("Implement me!")

<details>
    <summary><b>Solution</b></summary>
    
```python
def log_marginal_prob(x: list, phi: dict, pi: dict):
    """Return log P(X=x) under the model"""
    # **SOLUTION**
    # Tips:
    # * compute the joint probability for x and each of the possible labels
    #   marginalise (sum)
    #   compute log
    # * or, compute all of it in log space using log_joint_prob
    #   and np.logaddexp.reduce
    
    # for the marginalisation, we need the support of the rv Y, 
    # we can find it within the dictionary phi (ie, the set of keys in it)
    support_Y = phi.keys()
    # the function logaddexp(a, b) returns log(exp(a) + exp(b))
    # the method reduce applies that in a row to a whole list of values 

    return np.logaddexp.reduce([log_joint_prob(y=y, x=x, phi=phi, pi=pi) for y in support_Y]) 
```
    
---    
</details>

In [ ]:
# Your code should pass these tests

assert np.isclose(
    log_marginal_prob(x="not bad".split(), phi=test_phi_1, pi=test_pi_1), 
    np.log(0.6 * 0.05 * 0.15 + 0.4 * 0.05 * 0.5), 
    1e-3
)
assert np.isclose(
    log_marginal_prob(x="not good".split(), phi=test_phi_1, pi=test_pi_1), 
    np.log(0.6 * 0.05 * 0.5 + 0.4 * 0.05 * 0.1), 
    1e-3
)

**Exercise with solution** Impement a function to return $\log P_{Y|X}(y|x)$ for a label $y\in \mathcal Y$ and document $x \in \mathcal X$ under this model. Your function should use the functionality `log_prior_prob` and/or `log_conditional_prob` and/or `log_joint_prob` and/or `log_marginal_prob` from earlier. Assume the model parameters are already available. 

Below you will find some assertions (ie, test cases) for you to verify your implementation.

See API below:

In [ ]:
def log_posterior_prob(y: str, x: list, phi: dict, pi: dict):
    """Return log P(Y=y|X=x) under the model"""
    raise NotImplemented("Implement me !")

<details>
    <summary><b>Solution</b></summary>
    
```python
def log_posterior_prob(y: str, x: list, phi: dict, pi: dict):
    """Return log P(Y=y|X=x) under the model"""
    # **SOLUTION**
    
    
    # The posterior probability is a conditional probability
    #  by definition that is P(Y=y,X=x) / P(X=x)
    #  and we have already implemented the joint probability in the numerator
    log_joint_prob = log_joint_prob(y=y, x=x, phi=phi, pi=pi)
    #  and the marginal probability in the denominator
    log_marginal_prob = log_marginal_prob(x, phi=phi, pi=pi)
    # As we are in computing it in log space, we can subtract the logs to divide the probs
    return log_joint_prob - log_marginal_prob
```
    
---    
</details>

In [ ]:
# Your code should pass these tests

assert np.isclose(
    log_posterior_prob(y='pos', x="not bad".split(), phi=test_phi_1, pi=test_pi_1), 
    np.log((0.6 * 0.05 * 0.15)/(0.6 * 0.05 * 0.15 + 0.4 * 0.05 * 0.5)), 
    1e-3
)

assert np.isclose(
    log_posterior_prob(y='neg', x="not bad".split(), phi=test_phi_1, pi=test_pi_1), 
    np.log((0.4 * 0.05 * 0.5)/(0.6 * 0.05 * 0.15 + 0.4 * 0.05 * 0.5)), 
    1e-3
)

# The posterior is a distribution, thus it must add to one if we sum it over the possible labels:
assert np.isclose(
    np.exp(log_posterior_prob(y='pos', x="not bad".split(), phi=test_phi_1, pi=test_pi_1)) 
    + np.exp(log_posterior_prob(y='neg', x="not bad".split(), phi=test_phi_1, pi=test_pi_1)), 
    1, 
    1e-3
)

**Exercise with solution** Impement a function to return the most probable class under this model for a given document $x \in \mathcal X$. Your function should use the functionality `log_prior_prob` and/or `log_conditional_prob` and/or `log_joint_prob` and/or `log_marginal_prob` from earlier. Assume the model parameters are already available. 

Below you will find some assertions (ie, test cases) for you to verify your implementation.

See API below:

In [ ]:
def classify(x: list, phi: dict, pi: dict):
    """
    Classify x using the most probable label a-posteriori.
    
    x: a single document to be classified (a list of tokens)
    phi: parameters of the prior
    pi: parameters of the class-conditioned distributions over vocabulary
    
    Return the most probable label for the document
    """
    raise NotImplemented("Implement me!")

<details>
    <summary><b>Solution</b></summary>
    
```python
def classify(x: list, phi: dict, pi: dict):
    """
    Classify x using the most probable label a-posteriori.
    
    x: a single document to be classified (a list of tokens)
    phi: parameters of the prior
    pi: parameters of the class-conditioned distributions over vocabulary
    
    Return the most probable label for the document
    """
    # **EXERCISE**
    # Tips
    # * for each y, evaluate log p(s, y), then select the one that has highest value
    #   note that we have provided a helper method self.log_joint_prob(y, x)

    support_Y = list(phi.keys())
    best_k = np.argmax([log_joint_prob(y=y, x=x, phi=phi, pi=pi) for y in support_Y])
    y_pred = support_Y[best_k]
    return y_pred   
```
    
---    
</details>

In [ ]:
# Your code should pass these tests

# documents with single word
assert classify("good".split(), phi=test_phi_1, pi=test_pi_1) == 'pos'
assert classify("bad".split(), phi=test_phi_1, pi=test_pi_1) == 'neg'

# documents with multiple words
assert classify("so good".split(), phi=test_phi_1, pi=test_pi_1) == 'pos'
assert classify("so bad".split(), phi=test_phi_1, pi=test_pi_1) == 'neg'

# NBC is not good to handle modifiers, because it sees documents as unordered collections
assert classify("not good".split(), phi=test_phi_1, pi=test_pi_1) == 'pos'
assert classify("not bad".split(), phi=test_phi_1, pi=test_pi_1) == 'neg'

Hopefully it is clear to you that 
* the model is a probability distribution
* the model manipulates simpler probability factors to assign probability to larger documents
* the model definition and its parameter estimation are two very different things (so far we have been implementing all sorts of probability queries without ever having to worry about how parameters are estimated -- in fact the parameters have been given to us through `test_phi_1` and `test_pi_1`

We can now turn to parameter estimation via MLE, for which I need to give you a dataset:

**Exercise with solution** Implement a function that returns a dictionary of prior parameters (something similar to the `test_phi_1` object that we have been using. The numerical values of the parameters should be given by maximum likelihood estimation using a dataset of labelled documents.

Below you will find some assertions (ie, test cases) for you to verify your implementation.

See API below:

In [ ]:
def estimate_phi(data_x, data_y):
    """
    Return the dict that stores the parameters \phi of the prior.
    data_x: list of documents, each document is a list of tokens, each token is a string
    data_y: list of labels, each label is a string
    
        Documents and labels are paired, that is zip(data_x, data_y) will give you (x,y) pairs.
        
    Return a dict that maps from label to its probability mass using MLE as estimation algorithm.
    """
    raise NotImplemented("Implement me!")

<details>
    <summary><b>Solution</b></summary>
    
```python
from collections import Counter, defaultdict
from itertools import chain

    
def estimate_phi(data_x, data_y):
    """
    Return the dict that stores the parameters \phi of the prior.
    data_x: list of documents, each document is a list of tokens, each token is a string
    data_y: list of labels, each label is a string
    
        Documents and labels are paired, that is zip(data_x, data_y) will give you (x,y) pairs.
        
    Return a dict that maps from label to its probability mass using MLE as estimation algorithm.
    """
    # **SOLUTION**
    N = len(data_y)
    phi = dict((y, float(n) / N) for y, n in Counter(data_y).items())
    return phi 
```
    
---    
</details>

In [ ]:
# Your code should pass these tests

test_data_x_1 = [
    "pretty good".split(),
    "pretty good".split(),
    "pretty good".split(),
    
    "quite good".split(),
    "quite okay".split(),
    
    "so good".split(),
    "so pretty good".split(),
    
    "bad".split(),
    "bad bad bad".split(),
    "so very bad".split(),    
    "just so bad".split(),
    "bad just really really bad".split(),
]
test_data_y_1 = [
    'pos', 'pos', 'pos',
    'pos', 'pos',
    'pos', 'pos',
    'neg', 'neg', 'neg', 'neg', 'neg'
]

test_result_phi_1 = estimate_phi(test_data_x_1, test_data_y_1)

# First of all, we get a proper cpd
assert validate_categorical_cpd(test_result_phi_1)

# And it contains the probs we expect
assert np.isclose(test_result_phi_1['pos'], 7./(7+5), 0.01)
assert np.isclose(test_result_phi_1['neg'], 5./(7+5), 0.01)

# Let's use our helper code
assert get_prior_parameter('neu', test_result_phi_1) == 0.

**Exercise with solution** Implement a function that returns a dictionary of dictionaries for the parameters of the class-conditioned distributions over vocabulary (something similar to the `test_pi_1` object that we have been using). The numerical values of the parameters should be given by maximum likelihood estimation using a dataset of labelled documents.

Below you will find some assertions (ie, test cases) for you to verify your implementation.

Your code should support Laplace smoothing, but you can implement it first without (see you can pass the first set of tests, which does not depend on smoothing) and then try to modify it to use smoothing (and see if you can pass the second set of tests).


We provide some helper code, for determining the vocabulary from data (the vocabulary should contain some special symbols, e.g.  end-of-sequence  symbol, and, if Laplace smoothing is on, also a placeholder for future unseen tokens).

**Tip.** Students have found the Laplace smoothing procedure tricky in the past, so if you are spending too much time on it because your new to programming, it's okay to just study the solution. That said, try to get the non-smoothed version right, as that one is relatively simple.

See API below:

In [ ]:
from itertools import chain

def get_labels(data_y):
    """Return the set of labels in an observed dataset"""
    return set(data_y)

def get_vocabulary(data_x, alpha=0., EOS='-EOS-', UNK='-UNK-'):
    """
    Return the set of known words in an observed collection of documents. 
    To those we add the EOS symbol which needs to be in the support of the distribution, otherwise
     the Naive Bayes model does not have a 'stop criterion'.
    And, possibly, a placeholder for future unknown words, when we are using Laplace smoothing.
    """
    vocab = set(chain(*data_x))
    vocab.add(EOS) # should always be part of the vocabulary
    if alpha > 0.:  # we are reserving mass for future unseen words
        vocab.add(UNK)
    return vocab

def estimate_pi(data_x, data_y, alpha=0., EOS='-EOS-', UNK='-UNK-'):
    """
    Return the dict that stores the parameters of the class-conditioned distributions over vocabulary.
    
    data_x: list of documents, each document is a list of tokens, each token is a string
    data_y: list of labels, each label is a string
    
        Documents and labels are paired, that is zip(data_x, data_y) will give you (x,y) pairs.
    alpha: the Laplace smoothing coefficient, 
     a virtual count that gets added to every outcome that has been seen
     including all outcomes that have not been seen 
     (at this point we cannot know what new tokens will be seen in the future, 
      so we will be using a placeholder token for that, the special UNK token)
    EOS: a special token to be used as the end-of-sentence marker
     which you should pretend occurs at the end of every document
    UNK: a special token to be used as a placeholder for all unseen tokens
     this will only be used if you enable Laplace smoothing  (alpha > 0)  
        
    Return a dict that maps from label to its probability mass using MLE as estimation algorithm.
    """
    if alpha < 0:
        raise ValueError("Laplace smoothing requires a positive alpha")
        
    support_Y = get_labels(data_y) # all classes in the corpus
    vocab = get_vocabulary(data_x, alpha=alpha, EOS=EOS, UNK=UNK)
    V = len(vocab)

    raise NotImplemented("Implement me!")

<details>
    <summary><b>Solution</b></summary>
    
```python
from collections import Counter, defaultdict


def estimate_pi(data_x, data_y, alpha=0., EOS='-EOS-', UNK='-UNK-'):
    """
    Return the dict that stores the parameters of the class-conditioned distributions over vocabulary.
    
    data_x: list of documents, each document is a list of tokens, each token is a string
    data_y: list of labels, each label is a string
    
        Documents and labels are paired, that is zip(data_x, data_y) will give you (x,y) pairs.
    alpha: the Laplace smoothing coefficient, 
     a virtual count that gets added to every outcome that has been seen
     including all outcomes that have not been seen 
     (at this point we cannot know what new tokens will be seen in the future, 
      so we will be using a placeholder token for that, the special UNK token)
    EOS: a special token to be used as the end-of-sentence marker
     which you should pretend occurs at the end of every document
    UNK: a special token to be used as a placeholder for all unseen tokens
     this will only be used if you enable Laplace smoothing  (alpha > 0)  
        
    Return a dict that maps from label to its probability mass using MLE as estimation algorithm.
    """
    # **SOLUTION**
    
    if alpha < 0:
        raise ValueError("Laplace smoothing requires a positive alpha")
        
    support_Y = get_labels(data_y) # all classes in the corpus
    vocab = get_vocabulary(data_x, alpha=alpha, EOS=EOS, UNK=UNK)
    V = len(vocab)

    # Counts for (X,Y)
    joint_counts = dict((y, Counter()) for y in support_Y)
    for x, y in zip(data_x, data_y):
        counter_w_given_y = joint_counts[y]
        counter_w_given_y.update(x + [EOS])  # we gotta pad our sentences with EOS        
        
    # Conditional parameters pi
    pi = dict()
    
    for y in support_Y:
        counts_w_given_y = joint_counts[y]
        cpd = defaultdict(float)
        total_counts = sum(counts_w_given_y.values())
        for w in vocab:
#         for w, n in counts_w_given_y.items():  # renormalise (dealing with virtual counts as well)
            cpd[w] = (counts_w_given_y.get(w, 0) + alpha) / (total_counts + alpha * V)
        pi[y] = cpd
    
    return pi
```
    
---    
</details>

These are the tests for non-smoothed models ($\alpha=0$).

In [ ]:
test_result_pi_1 = estimate_pi(test_data_x_1, test_data_y_1)

# First of all we get proper cpds
assert all(validate_categorical_cpd(cpd) for y, cpd in test_result_pi_1.items())

# There are 8 occurrences of 'bad' in negative documents
# and putting all negative documents together we have 20 words (already taking EOS tokens into account)
assert np.isclose(test_result_pi_1['neg']['bad'], 8/20, 0.01)

# There are 2 occurrences of 'so' in negative documents
# and putting all negative documents together we have 20 words (already taking EOS tokens into account)
assert np.isclose(test_result_pi_1['neg']['so'], 2/20, 0.01)

# There are 0 occurrences of 'good' in negative documents
# and putting all negative documents together we have 20 words (already taking EOS tokens into account)
assert np.isclose(test_result_pi_1['neg']['good'], 0/20, 0.01)

# There are 2 occurrences of 'so' in positive documents
# and putting all negative documents together we have 20 words (already taking EOS tokens into account)
assert np.isclose(test_result_pi_1['pos']['so'], 2/22, 0.01)

# EOS artificially occurs once per document
assert np.isclose(test_result_pi_1['neg']['-EOS-'], 5/20, 0.01)
assert np.isclose(test_result_pi_1['pos']['-EOS-'], 7/22, 0.01)

#assert np.isclose(test_result_pi_1['neg']['-EOS-'], 1/((2+4+4+4+6)/5), 0.01)
#assert np.isclose(test_result_pi_1['pos']['-EOS-'], 1/((3+3+3+3+3+3+4)/7), 0.01)

# No mass reserved for future unseen words
assert get_cond_parameter(w="alright", y="pos", pi=test_result_pi_1) == 0.

For smoothed models (with $\alpha>0$) we need to change our definition of `get_cond_parameter` so that it uses the probability of the UNK token when it finds an unseen token.

In [ ]:
def get_cond_parameter(y: str, w: str, pi: dict, UNK='-UNK-'):
    """
    Return P(W=w|Y=y) under the model Categorical(\pi^{(y)}).
    
    y: the label
    w: word
    pi: a dictionary of dictionaries 
        first we can index it using a label to obtain a dict as return, 
        the latter dict are the parameters for a distribution over the vocabulary given the label
        and it can be indexed using a word to obtain a probability mass

        even though mathematically we think of pi as a table/matrix
        in code it can be convenient to treat it like a dictionary some times, 
        for example, as a dictionary we can use labels that are strings (rather than 0-based indices)
        and words that are strings (rather than 0-based indices to a vocabulary).        
        
        For this implementation assume that every dict inside of pi
         has already been validated as a Categorical parameter.
    UNK: we return the probability P(W=UNK|Y=y) in case w is not in the support.
        
    Return: Categorical(w|\phi^{(y)})
    """
    cpd = pi.get(y, None)
    if cpd is None:  # the label is not in the support of the model
        return 0. 
    # we fetch the probability of the UNK symbol
    # and use it as default for when we fetch the probability of the word w
    return cpd.get(w, cpd.get(UNK, 0.))  # if a word is outside the support, it gets 0 mass

Now you can test with Laplace smoothing on:

In [ ]:
test_result_pi_2 = estimate_pi(test_data_x_1, test_data_y_1, alpha=1.)

In [ ]:
# First of all we get proper cpds
assert all(validate_categorical_cpd(cpd) for y, cpd in test_result_pi_2.items())

In [ ]:
# The first 7 documents are positive:
#  number of words in positive docs 
#  + number of occurrences of EOS 
#  + one virtual count per word in vocab (which includes UNK and EOS)
assert sum(len(x) for x in test_data_x_1[:7]) + 7 + 1 * len(get_vocabulary(test_data_x_1, alpha=1.0)) == 33
# Seen word, but not seen with this label
assert get_cond_parameter(w="really", y="pos", pi=test_result_pi_2) == 1/33
# Unseen word
assert get_cond_parameter(w="alright", y="pos", pi=test_result_pi_2) == 1/33

In [ ]:
# The last 5 documents are negative:
#  number of words in neagtive docs 
#  + number of occurrences of EOS 
#  + one virtual count per word in vocab (which includes UNK and EOS)
assert sum(len(x) for x in test_data_x_1[-5:]) + 5 + 1 * len(get_vocabulary(test_data_x_1, alpha=1.0)) == 31
# Seen word, but not seen with this label
assert get_cond_parameter(w="quite", y="neg", pi=test_result_pi_2) == 1/31
# Unseen word
assert get_cond_parameter(w="alright", y="neg", pi=test_result_pi_2) == 1/31

**Congratulations! You've implemented every aspect of an NB classifier :D**

The rest of this section will just organise the code and put it for you in a single "package", so that you can use it in an experiment (which you will conduct in the next section).

This section also contains a small demonstration of how you can use the newly developed NB classifier.


---


When we have a number of related functionalities (e.g., get parameters, combine them into joint probabilities, marginal probabilities, use them for predictions, code for estimating the parameters themselves, etc.), it is convenient to group all these functionalities into something like a container.

This is called a *class* in many programming languages. Think of classes as templates for instantiating objects that store data and code for you to reuse over and over in different situations.

You don't need to be able write a class yourself, but it is useful to study the one below. 

In this course, programming skills *are not* assessed in exams, so studying a class definition and learning to use such a concept is a skill that will help your programming and it is a practical skill of great importance.

In [ ]:
import numpy as np
from collections import defaultdict, Counter
from itertools import chain, takewhile


class NaiveBayesClassifier:
    
    def __init__(self, labels: list, vocab=[], alpha=0., EOS='</s>', UNK='<unk>', seed=None):
        """
        This constructs a NaiveBayesClassifier built upon a joint distribution over X and Y
        where X are documents and Y is a label.
        
        The set of labels is finite and known ahead of time.

        Every document X is a sequence of tokens, a token W is a word in a finite vocabulary.
        We can start with a given vocabulary if we already know it, 
         but in any case, this class offers a training procedure which will use the words in a given corpus
         as the set of known words in the vocabulary.
         
        The NBC needs two special symbols for practical purposes. The end-of-sequence symbol (EOS)
         helps the NBC stop generating, when the model is used for generation of new documents.
         The UNK token is a placeholder for every unseen word we may encounter in the future.
         
        Without training, this NBC uses uniform cpds for every cpd in the model

        With training, this NBC may uses Laplace smoothing (if you set alpha to something greater than 0.).
        
        After the class definition you will find a demonstration of how to use it.
        
        :param labels: a list of classes, each a string or integer
        :param alpha: the smoothing coefficient for distributions of the kind X|Y=y
        :param EOS: the EOS symbol
        :param UNK: the UNK symbol for smoothing
        :param seed: random generator seed, fix this for reproducibility
        """
        self._EOS = EOS
        self._UNK = UNK
        self._alpha = alpha        
        self._labels = tuple(labels)
        self._rng = rng = np.random.RandomState(seed)  # Good for reproducibility
        # The method fit will populate these
        self._vocab = set(vocab)
        self._vocab.add(EOS)
        self._vocab.add(UNK)
        # we initialise our distributions with uniform probabilities
        #  but we will use a trick, if these objects are None, 
        #  the methods that read them will return a uniform probability by default
        #  this way we do not need to do anything at this point
        self._prior_probs = None
        # intialise with nothing
        #  we will use a trick, whenever we have None, the probabilities are going to be uniform
        self._cond_probs = None    
        
    def get_prior_parameter(self, y: str):
        """
        Return P(Y=y) = phi[y]
        
        The only difference compared to before is that this time we assume a uniform distribution
        when the model is untrained.
        """
        return 1/len(self._labels) if self._prior_probs is None else self._prior_probs[y]    
    
    def get_cond_parameter(self, y: str, w: str):
        """
        Return P(W=w|Y=y) = pi[y, w]
        
        If the model is untrained, we assume the probaiblity is uniform over the vocabulary.
        """
        if self._cond_probs is None:
            # return uniform probability if the model isn't trained
            return 1/len(self._vocab) 
        else:
            # prob of unk|y
            unk_prob = self._cond_probs[y].get(self._UNK, 0.0)
            # return w|y if w is in the dictionary, if not return unk|y
            return self._cond_probs[y].get(w, unk_prob)     
        
    def log_prior_prob(self, y: str):
        """Return log P(Y=y)"""
        return np.log(self.get_prior_parameter(y))
        
    def log_conditional_prob(self, x: list, y: str):
        """
        Return log P(X=x|Y=y)
        """
        return sum(np.log(self.get_cond_parameter(y=y, w=w)) for w in x)  # named arguments help us not make mistakes
    
    def log_joint_prob(self, y: str, x: list):
        """Return log P(Y=y) + log P(X=x|Y=y)"""
        return self.log_prior_prob(y) + self.log_conditional_prob(x=x, y=y)  # named arguments help us not make mistakes
     
    def log_marginal_prob(self, x: list):
        """Return log P(X=x) = log \sum_y P(Y=y, X=x)"""
        # the function logaddexp(a, b) returns log(exp(a) + exp(b))
        # the method reduce applies that in a row to a whole list of values 
        return np.logaddexp.reduce([self.log_joint_prob(y=y, x=x) for y in self._labels])  # named arguments help us not make mistakes
    
    def log_posterior_prob(self, y: str, x: list):
        """Return log P(Y=y|X=x) under the model"""

        # The posterior probability is a conditional probability
        #  by definition that is P(Y=y,X=x) / P(X=x)
        #  and we have already implemented the joint probability in the numerator
        log_joint = self.log_joint_prob(y=y, x=x)
        #  and the marginal probability in the denominator
        log_marginal = self.log_marginal_prob(x)
        # As we are in computing it in log space, we can subtract the logs to divide the probs
        return log_joint - log_marginal
    
    def predict(self, data_x: list):
        """
        :param data_x: a list of sentences to be classified, each sentence is a list of tokens (each token is a string)

        :return: a list of predictions, one per sentence in data_x, 
            each prediction is the label with highest posterior probability given the input
        """        
        y_pred = []
        for x in data_x:            
            # the label with highest joint probability with x is also 
            # the label with the highest posterior probability (since P(X=x) is fixed)
            # also, we can do everything in log domain
            best_k = np.argmax([self.log_joint_prob(y=y, x=x) for y in self._labels])
            y_pred.append(self._labels[best_k])
        return y_pred  
    
    def fit(self, data_x: list, data_y: list):
        """
        Return the dict that stores the parameters of the class-conditioned distributions over vocabulary.

        data_x: list of documents, each document is a list of tokens, each token is a string
        data_y: list of labels, each label is a string

            Documents and labels are paired, that is zip(data_x, data_y) will give you (x,y) pairs.
        alpha: the Laplace smoothing coefficient, 
         a virtual count that gets added to every outcome that has been seen
         including all outcomes that have not been seen 
         (at this point we cannot know what new tokens will be seen in the future, 
          so we will be using a placeholder token for that, the special UNK token)
        EOS: a special token to be used as the end-of-sentence marker
         which you should pretend occurs at the end of every document
        UNK: a special token to be used as a placeholder for all unseen tokens
         this will only be used if you enable Laplace smoothing  (alpha > 0)  

        Return a dict that maps from label to its probability mass using MLE as estimation algorithm.
        """
        
        # Make the set of classes
        support_Y = self._labels
        assert set(self._labels) == set(data_y) # we need observations for all classes 
        
        # Make the vocabulary of words
        # Vocabulary (support of W is all known words)
        self._vocab = set(chain(*data_x))
        # plus the special EOS token
        self._vocab.add(self._EOS)
        if self._alpha > 0.:
            # and the special UNK token
            self._vocab.add(self._UNK)
        V = len(self._vocab)

        # MLE for Y
        N = len(data_y)
        # count_Y(y) divided by total data points N (no need for smoothing here)
        self._prior_probs = dict((y, float(n) / N) for y, n in Counter(data_y).items())
        
        # MLE for W|Y
        # Counts for (X,Y)
        joint_counts = dict((y, Counter()) for y in support_Y)
        for x, y in zip(data_x, data_y):
            counter_w_given_y = joint_counts[y]
            counter_w_given_y.update(x + [self._EOS])  # we gotta pad our sentences with EOS        

        # Conditional parameters
        self._cond_probs = dict()

        for y in support_Y:
            counts_w_given_y = joint_counts[y]
            cpd = defaultdict(float)
            total_counts = sum(counts_w_given_y.values())
            for w in self._vocab:
                cpd[w] = (counts_w_given_y.get(w, 0) + self._alpha) / (total_counts + self._alpha * V)
            self._cond_probs[y] = cpd
    
    def sample_n(self, num_samples=1, y=None, max_length=200):
        """
        Sample a number of times from the joint distribution, or from the conditional
        :param num_samples: how many samples do we want to generate
        :param y: if given, sample from S|Y=y, otherwise from Y and then from S|Y    
        :param max_length: maximum length for s
        :return: a generator of samples, each a pair (y, x)
        """   
        
        # This algorithm is a bit tricky, 
        #  you do not need to study it necessarily
        # Here's what it does conceptually
        # for _ in range(S)
        #  draw Y from Cat(phi)
        #  draw W[i]|Y=x from Cat(pi[y]) until we draw an EOS symbol
        # But, to make it a bit more efficient, we draw
        #  S times from Cat(phi)
        # Then for each of those draws s=1, ..., S we draw
        #  L times from Cat(pi[y[s]]) where L is max length
        #  we then find the position of the first EOS and discard from there onwards
        
        if y is None: # Draw num_samples times from P_Y
            # np.random.choice returns an integer in [0, a) and p is the discrete probability distribution it samples from
            # we obtain num_samples such samples
            ids = self._rng.choice(len(self._labels), p=[self.get_prior_parameter(k) for k in self._labels], size=num_samples)
            # these are the sampled labels
            sampled_labels = [self._labels[k] for k in ids]
        else:  # or repeat the given label num_samples times
            # here the sampled_labels are whatever the user chose via argument y
            sampled_labels = [y for _ in range(num_samples)]

        # here we turn the support of the conditionals into a list for easy access
        vocab = [w for w in self._vocab]
        vocab_size = len(vocab)        
        # here we turn the conditionals into np arrays so it's easier to sample from
        probs = dict()        
        for label in set(sampled_labels):
            probs[label] = np.array([self.get_cond_parameter(y=label, w=w) for w in self._vocab])

        # for each sampled label, we also sample a document
        for label in sampled_labels:
            # I know use numpy to sample max_length tokens following the distribution W|Y=y
            # I also map numpy's choices (which are indices) to words using `vocab`
            seq = [vocab[v] for v in self._rng.choice(vocab_size, max_length, p=probs[label])]
            # Finally, I throw away whatever happens after the first occurrence of EOS
            # since the sampler should have stopped at that point
            # (this wastes a bit of computation, but it's okay for our demonstration)
            yield (label, list(takewhile(lambda w: w != self._EOS, seq)) )   

Here is a **toy demonstration** to help you with debugging your code.

In [ ]:
# an untrained model with 
toy_nbc = NaiveBayesClassifier(['neg', 'pos'], vocab=['pepper', 'the', 'dog'], alpha=0.01, seed=23)

See that you can sample from an untrained model. That's because an untrained model *is* a probability distribution nonetheless. It's just a not very good one.

In [ ]:
for y, x in toy_nbc.sample_n(10):
    print('{}: {}'.format(y, ' '.join(x)))

Here's a demonstration of how to train the model:

In [ ]:
toy_nbc.fit(
    [
        'this is bad'.split(), 
        'this is really good'.split(),
        'this is pretty good'.split(),
        'this is great'.split(),
    ], ['neg', 'pos', 'pos', 'pos']
)

In [ ]:
assert toy_nbc._prior_probs == {'neg': 0.25, 'pos': 0.75},  "Note the toy example has 1 negative and 3 positive instances, so we expect 1/4 and 3/4 as probability, not: {}".format(toy_nbc._phi)

In [ ]:
assert toy_nbc._vocab == {'</s>', '<unk>', 'bad', 'good', 'great', 'is', 'pretty', 'really', 'this'}, "Did you perhaps forget to create a vocab or to add the UNK and EOS tokens?" 

In [ ]:
assert toy_nbc.get_cond_parameter(y='neg', w='pretty') == toy_nbc.get_cond_parameter(y='neg', w='<unk>'), "The word 'pretty' was never seen by"

Note that the distributions should normalise over the complete sample space (which includes EOS and UNK):

In [ ]:
assert np.isclose(sum(toy_nbc.get_cond_parameter(y='neg', w=w) for w in toy_nbc._vocab), 1, 1e-3), "Did you smooth things correctly? Try again with alpha=0.0, then see if you can fix it for alpha > 0.0"

In [ ]:
assert np.isclose(sum(toy_nbc.get_cond_parameter(y='pos', w=w) for w in toy_nbc._vocab), 1, 1e-3), "Did you smooth things correctly? Try again with alpha=0.0, then see if you can fix it for alpha > 0.0"

Now you can use the model for making predictions

In [ ]:
toy_pred = toy_nbc.predict(
    [
        'this is really bad'.split(), 
        'this is good'.split(), 
        'this is pretty'.split()
    ]
)
assert toy_pred == ['neg', 'pos', 'pos'], "Something seems to have gone wrong with your predictions"

and for computing probabilities:

In [ ]:
assert toy_nbc.log_conditional_prob(x='this is really bad'.split(), y='neg') > toy_nbc.log_conditional_prob(x='this is really bad'.split(), y='pos'), "The negative LM likes this sentence more"

In [ ]:
assert toy_nbc.log_conditional_prob(x='this is really bad'.split(), y='neg') >  toy_nbc.log_marginal_prob(x='this is really bad'.split()) > toy_nbc.log_conditional_prob('this is really bad'.split(), y='pos'), "The marginal probability is in between"

You can even sample from the generative story of the model. 

See how about 3/4 of the samples will be positive:

In [ ]:
assert np.isclose(sum(1 for y, s in toy_nbc.sample_n(1000) if y == 'pos')/1000, 0.75, 0.1)

In [ ]:
# Random generation:
for y, x in toy_nbc.sample_n(10):
    print('{}: {}'.format(y, ' '.join(x)))
    
print('\nOut of 100 samples we have {} positive.'.format(sum(1 for y, s in toy_nbc.sample_n(100) if y == 'pos')))

We can also choose to sample conditionally, in this case, we ask for samples from the *positive* class only:

In [ ]:
for y, s in toy_nbc.sample_n(10, y='pos'):
    print('{}: {}'.format(y, ' '.join(s)))

print('\nAverage length in 100 positive samples is {:.2f}'.format(sum(len(s) for y, s in toy_nbc.sample_n(100, y='pos'))/100.))    

Same can be done for the negative class:

In [ ]:
for y, s in toy_nbc.sample_n(10, y='neg'):
    print('{}: {}'.format(y, ' '.join(s)))

print('\nAverage length in 100 positive samples is {:.2f}'.format(sum(len(s) for y, s in toy_nbc.sample_n(100, y='neg'))/100.))    

And of course, we can check posterior probabilities:

In [ ]:
np.exp(toy_nbc.log_posterior_prob(x="too bad".split(), y="neg")) 

In [ ]:
np.exp(toy_nbc.log_posterior_prob(x="too bad".split(), y="pos"))

The NB classifier *supports* actual documents, that is, it assigns non-zero probability to actual documents. But it also supports plenty of sequences that are not at all plausible, even after training. This may seem strange at first, but it is actually to be expected, given that it makes too strong assumptions (it is a "naive" model after all).

This does not mean the model is bad. Generating documents is not its primary *application*, even though being able to generate a document is part of its fundamental design. We call it a "document", but for the model it is just a collection of words that are considered related to classes, theses words are not in order, and the model has no clue what these words mean for a human. 

The primary application of NBC is to represent our uncertainty about the categorisation of a document, so its primary application are inferences we make based on the posterior distribution (or the joint distribution, depending on the purpose).  In the next section we will use the model for classification. 

We will also play with it as a sampler, just to see what we get, but don't hope for much yet :)

## <a name="exp">  Experiment


We will now try our NBC on one of NLTK's datasets.

As usual in testing a machine learning model, we will need training data, development data, and test data. Check [Section 4.8](https://web.stanford.edu/~jurafsky/slp3/4.pdf) for a recap of how to train/test a machine learning classifier. There's no need to implement cross-validation (a training/development/test split is sufficient, and we provide helper code for splitting the data).

In [ ]:
import numpy as np
from itertools import chain


def split_corpus(sentences, ratio=0.9):
    """
    Randomly split a list of sentences into two sets according to the given ratio.
    
    :param sentences: already tokenized sentences (list of strings)
    """
    # This will guarantee that the permutation is the same every time (which is important for reproducibility)
    rng = np.random.RandomState(42)    
    indices = rng.permutation(len(sentences))
    n = int(indices.size * ratio)
    return [sentences[i] for i in indices[:n]], [sentences[i] for i in indices[n:]]

Let's attempt to predict sentence polarity

In [ ]:
from nltk.corpus import sentence_polarity

pos_sents = sentence_polarity.sents(categories='pos')
neg_sents = sentence_polarity.sents(categories='neg')
print(len(pos_sents), 'positive sentences such as:\n', ' '.join(pos_sents[0]))
print(len(neg_sents), 'negative sentences such as:\n', ' '.join(neg_sents[0]))

And here is a reasonable split:

In [ ]:
training_pos, dev_pos = split_corpus(pos_sents, 0.8)
dev_pos, test_pos = split_corpus(dev_pos, 0.5)
training_neg, dev_neg = split_corpus(neg_sents, 0.8)
dev_neg, test_neg = split_corpus(dev_neg, 0.5)

In [ ]:
rows = [
    ['pos', len(training_pos), len(dev_pos), len(test_pos)],
    ['neg', len(training_neg), len(dev_neg), len(test_neg)]
]
print(tabulate(rows, headers=['Label', 'Training', 'Development', 'Test']))

<a name="ex-NBC"> **Graded Exercise - NBC for sentence polarity** 
    
Fit NBC, evaluate it on dev set. To recap how you evaluate a classifier, check [Section 4.7](https://web.stanford.edu/~jurafsky/slp3/4.pdf). 

For this use $\alpha=1.0$. For evaluation you can use `sklearn.metrics import classification_report`, check its documentation. You can install scikit-learn via `!pip install sklearn`.

In [ ]:
# CONTRIBUTE YOUR SOLUTION

<a name="ex-grid"> **Graded Exercise - Grid Search** 
    
Grid-search for a good value of $\alpha$, use the grid shown below. Use performance on *development* set to pick the best value of the hyperparameter. If your data are not balanced (e.g., brown) you should choose the precise metric carefully (not every metric in scikit's evaluation report is equally good with class imbalance). For the best value of $\alpha$ then test your NBC on the test-set.

In [ ]:
grid_alpha = 0.1 + np.arange(0., 1.5, 0.1)
grid_alpha

In [ ]:
# CONTRIBUTE YOUR SOLUTION

## Error analysis 

This is a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix), it displays a summary of our predictions in terms of true positives (TP), false positive (FP), false negative (FN), and true negatives (TN). 

<a name="ex-analysis"> **Graded Exercise - Error analysis** 
    
Print the confusion matrix of your classifier using the dev set, use $\alpha=0.1$ for training. 

You can build the matrix yourself, or you can use sklearn's `from sklearn.metrics import confusion_matrix`. 

Then, manually inspect some of the mistakes (FP or FN) and speculate about directions for improvement in the future, in particular, find (and list) examples of **two or more** patterns that the model fails to recognise **because** of its assumptions. Explain how you think the model assumptions are responsible for the failure.

In [ ]:
# CONTRIBUTE YOUR SOLUTION

# <a name="further"> Further analysis



**Ungraded exercise without solution** Draw samples from this NBC model (use as many samples as you have data points in total).

Investigate the following:

* label frequency
* sequence length
* sequence length per class
* most frequent words per class


What aspects of the observed data do you think model samples reflect well?

What aspects do they not?

This exercise is here just to help you visualise some consequences of our modelling assumptions so far. It is not part of the tutorial per se. It connects T1 and T2, so in that sense we recommend you look into this, for example, after you are done with your graded assignments, there will be more discussion on this topic in class. 